In [ ]:
import pandas as pd
import numpy as np
from google.cloud.bigquery import Client, QueryJobConfig
import seaborn as sns

In [ ]:
data = pd.read_csv('data_all_merged_03_08_23_vfinal.csv')
data = data.iloc[:,1:]

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """
SELECT *
FROM
  (
  SELECT 
    fo_prod_table.yeti_customer_uuid,
    fo_prod_table.first_order_id, 
    fo_prod_table.price,
    fo_prod_table.product_category,
    DENSE_RANK() OVER(PARTITION BY fo_prod_table.yeti_customer_uuid ORDER BY fo_prod_table.price DESC) as rnk1
  FROM
    (
    SELECT 
      fo_id.yeti_customer_uuid,
      fo_id.first_order_id, 
      oi_prod_table.product_id, 
      oi_prod_table.product_category, 
      oi_prod_table.price,
    FROM
      (SELECT
        orders.yeti_customer_uuid,
        COUNT(orders.order_id) as order_cnt,
        MIN(orders.order_id) as first_order_id,
        MIN(orders.creation_date) first_order_date,
      FROM `yeti-dev-edw.aa_anon.aa_order` orders
      WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
      GROUP BY orders.yeti_customer_uuid
      HAVING order_cnt > 1) fo_id
    LEFT JOIN
      (
      SELECT
        order_item.yeti_customer_uuid,
        order_item.order_id,
        order_item.order_item_id,
        order_item.product_id, 
        product.product_category, 
        product.price, 
      FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
      LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
      ON order_item.product_id = product.product_id) oi_prod_table
    ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
    ) fo_prod_table
  )
WHERE rnk1 = 1
"""

job = client.query(query)
df = job.to_dataframe()

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """
SELECT *
FROM
  (
  SELECT 
    fo_prod_table.yeti_customer_uuid,
    fo_prod_table.first_order_id, 
    fo_prod_table.price,
    fo_prod_table.product_category,
    fo_prod_table.product_spend,
    DENSE_RANK() OVER(PARTITION BY fo_prod_table.yeti_customer_uuid ORDER BY fo_prod_table.product_spend DESC) as rnk1
  FROM
    (
    SELECT 
      fo_id.yeti_customer_uuid,
      fo_id.first_order_id, 
      oi_prod_table.product_id, 
      oi_prod_table.product_category, 
      oi_prod_table.price,
      oi_prod_table.product_spend,
    FROM
      (SELECT
        orders.yeti_customer_uuid,
        COUNT(orders.order_id) as order_cnt,
        MIN(orders.order_id) as first_order_id,
        MIN(orders.creation_date) first_order_date,
      FROM `yeti-dev-edw.aa_anon.aa_order` orders
      WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
      GROUP BY orders.yeti_customer_uuid
      HAVING order_cnt > 1) fo_id
    LEFT JOIN
      (
      SELECT
        order_item.yeti_customer_uuid,
        order_item.order_id,
        order_item.order_item_id,
        order_item.product_id, 
        product.product_category, 
        product.price, 
        order_item.quantity*product.price as product_spend,
      FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
      LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
      ON order_item.product_id = product.product_id) oi_prod_table
    ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
    ) fo_prod_table
  )
WHERE rnk1 = 1
ORDER BY yeti_customer_uuid
"""

job = client.query(query)
df = job.to_dataframe()

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat[data_with_entry_prod_cat['more_than_1_entry_prod_cat?'] == 0]

In [ ]:
df.pivot_table(index = ['yeti_customer_uuid'], aggfunc ='size')


yeti_customer_uuid
0000095c-0e7f-4fe6-917e-1d2356aed1c4    1
00001699-04d3-45e8-aedf-6c2a1cdd57ea    2
00001dcc-dd92-49e9-b2a1-42cf8fd923fa    1
000023e6-98b2-4f5d-bc9d-6232e3eb244e    1
00003f8f-fdc9-4b04-860f-7b6e36bb8af5    1
                                       ..
ffffcdf6-70c6-43a7-b5c0-088eae60a06a    1
ffffe8b5-794e-4537-962a-00e88b920c3e    1
ffffedf7-056d-4fa8-836e-d805d90558fe    1
fffff759-bb94-4ce8-92bd-b156dd29de1b    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

In [ ]:
df[df['yeti_customer_uuid'] == '00001699-04d3-45e8-aedf-6c2a1cdd57ea']

,yeti_customer_uuid,first_order_id,price,product_category,product_spend,rnk1
2,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.0,Drinkware,25.0,1
3,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.0,Drinkware,25.0,1


In [ ]:
entry_product_cat_df = df[df.duplicated()==False]

In [ ]:
entry_product_cat_df

,yeti_customer_uuid,first_order_id,price,entry_prod_cat,product_spend,rnk1
0,None,100003602,NaN,None,NaN,1
1,0000095c-0e7f-4fe6-917e-1d2356aed1c4,INV-571256,38.00,Drinkware,38.00,1
2,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.00,Drinkware,25.00,1
4,00001dcc-dd92-49e9-b2a1-42cf8fd923fa,201989165,18.74,Drinkware,18.74,1
5,000023e6-98b2-4f5d-bc9d-6232e3eb244e,202173939,25.00,Drinkware,100.00,1
...,...,...,...,...,...,...
2692502,ffffcdf6-70c6-43a7-b5c0-088eae60a06a,203424722,40.00,Drinkware,40.00,1
2692503,ffffe8b5-794e-4537-962a-00e88b920c3e,203877222,250.00,Hard Cooler,250.00,1
2692504,ffffedf7-056d-4fa8-836e-d805d90558fe,202498224,37.49,Drinkware,899.76,1
2692505,fffff759-bb94-4ce8-92bd-b156dd29de1b,INV-318899,38.00,Drinkware,38.00,1


In [ ]:
data_with_entry_prod_cat.pivot_table(index = ['customer_id'],aggfunc = 'size')

customer_id
0000095c-0e7f-4fe6-917e-1d2356aed1c4    1
00001699-04d3-45e8-aedf-6c2a1cdd57ea    1
00001dcc-dd92-49e9-b2a1-42cf8fd923fa    1
000023e6-98b2-4f5d-bc9d-6232e3eb244e    1
00003f8f-fdc9-4b04-860f-7b6e36bb8af5    1
                                       ..
ffffcdf6-70c6-43a7-b5c0-088eae60a06a    1
ffffe8b5-794e-4537-962a-00e88b920c3e    1
ffffedf7-056d-4fa8-836e-d805d90558fe    1
fffff759-bb94-4ce8-92bd-b156dd29de1b    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

In [ ]:
data_with_entry_prod_cat[data_with_entry_prod_cat['customer_id']=='b7b75c1c-c906-446c-9f8c-f0e899618776']

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat,price
1613862,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Outdoor Living,300.0


In [ ]:
data_with_entry_prod_cat[['customer_id']].value_counts()

customer_id                         
b7b75c1c-c906-446c-9f8c-f0e899618776    3
bfebf243-c72f-4459-bad0-aec00b75a98a    3
a290f9f3-a2ed-49ae-b83a-74faffbcd1ef    3
5539a118-2b3f-4fc3-a47f-5a9081260597    3
a9a337af-f30e-4a37-bf8a-13debca27b13    3
                                       ..
55516972-7b1a-4651-b125-53311a8426d9    1
5551515d-6d7f-4e71-8cb6-975fda97f762    1
555148fe-9d58-4de9-957e-b8b979a58de0    1
5551470e-a7c8-43f8-abcd-01cd280d791c    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat,price
1613357,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Soft Cooler,300.0
1613358,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Outdoor Living,300.0
1613359,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Bags,300.0


In [ ]:
# data_with_entry_prod_cat.drop(['more_than_1_entry_prod_cat?','price'],axis=1)

In [ ]:
def make_entry_prod_cat_null(cust_id):
    if cust_id in cust_with_more_than_1_entry_prod_cat_list:
        return 1
    else:
        return 0

In [ ]:
data_with_entry_prod_cat['more_than_1_entry_prod_cat?']=data_with_entry_prod_cat['customer_id'].map(make_entry_prod_cat_null)

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat[data_with_entry_prod_cat['more_than_1_entry_prod_cat?'] == 0]

In [ ]:
# data_with_entry_prod_cat.drop(['more_than_1_entry_prod_cat?','price'],axis=1)

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat.iloc[:,:-2] # dropping more_than_1_entry_prod_cat? and price column

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].value_counts()

Drinkware         1618315
Hard Cooler        210632
Soft Cooler        113937
Other               60593
Bags                47296
Outdoor Living      38465
Cargo               14980
(not set)            2730
Name: entry_prod_cat, dtype: int64

In [ ]:
data_with_entry_prod_cat.columns

Index(['customer_id', 'months_elapsed', 'lifetime_spent', 'per_order_spent',
       'outdoor_equipment_pct', 'Cargo_pct', 'Soft_Cooler_pct', 'Bags_pct',
       'drinkware_pct', 'Hard_Cooler_pct', 'other_pct', 'purchase_times',
       'recency', 'avg_time_btw_orders', 'email_received', 'email_open_rate',
       'unsubscribed', 'months_since_last_email_open',
       'num_distinct_prod_cats', 'lifetime_quantity', 'per_order_quantity',
       'has_discount', 'black', 'navy', 'white', 'seafoam', 'stainless',
       'charcoal', 'nordic_purple', 'harvest_red', 'alpine_yellow',
       'other_color', 'yeti_customer_uuid', 'entry_prod_cat'],
      dtype='object')

In [ ]:
def convert_entry_prod_other_cats(prod_cat):
    if prod_cat == '(not set)':
        return 'Other'
    else:
        return prod_cat

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].map(convert_entry_prod_other_cats).value_counts()

Drinkware         1618315
Hard Cooler        210632
Soft Cooler        113937
Other               63323
Bags                47296
Outdoor Living      38465
Cargo               14980
Name: entry_prod_cat, dtype: int64

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].value_counts().sum()

2106948

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].isna().sum()

20307

In [ ]:
data_with_entry_prod_cat

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,white,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat
0,b32e89d5-a934-4d77-8b86-e01bf270fce6,83.700000,1771002.0,124.806342,0.000000,0.0,0.000000,0.000000,0.974359,0.025641,...,5.0,1.0,5.0,0.0,0.0,0.0,0.0,19.0,b32e89d5-a934-4d77-8b86-e01bf270fce6,Drinkware
1,20a5cbb9-216a-4bfb-84f8-34316cc990f0,64.566667,766093.0,194.637449,0.000000,0.0,0.055556,0.000000,0.944444,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,20a5cbb9-216a-4bfb-84f8-34316cc990f0,Drinkware
2,564fd988-bc35-475f-b7f6-da04ae474422,56.066667,910602.0,91.288421,0.047619,0.0,0.000000,0.000000,0.619048,0.047619,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,18.0,564fd988-bc35-475f-b7f6-da04ae474422,Drinkware
3,e6867db2-26c5-4841-8822-863bce76f5b0,69.266667,1137507.0,84.787343,0.125000,0.0,0.000000,0.000000,0.437500,0.000000,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,20.0,e6867db2-26c5-4841-8822-863bce76f5b0,Drinkware
4,36a6c5a8-955a-457e-8509-83b7e0644856,23.666667,2138149.0,386.995294,0.000000,0.0,0.176471,0.058824,0.588235,0.000000,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,9.0,36a6c5a8-955a-457e-8509-83b7e0644856,Drinkware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135824,c6882f0e-203b-41f7-bc5f-1f24713a3179,47.566667,3749.0,178.523810,0.000000,0.0,0.000000,0.000000,0.857143,0.142857,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,c6882f0e-203b-41f7-bc5f-1f24713a3179,Hard Cooler
2135825,1cb47d1e-ead0-4432-8d40-53ce0d3df887,86.700000,357405.0,120.826572,0.000000,0.0,0.000000,0.000000,0.857143,0.142857,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,1cb47d1e-ead0-4432-8d40-53ce0d3df887,Drinkware
2135826,3973bf5a-f1db-4a38-8ffb-bfd8a46a5027,58.966667,667411.0,64.728057,0.000000,0.0,0.000000,0.000000,0.857143,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3973bf5a-f1db-4a38-8ffb-bfd8a46a5027,Drinkware
2135827,902a88ec-2eee-444f-a524-c7f0e5f5c3bc,35.066667,243385.0,69.261525,0.000000,0.0,0.000000,0.000000,0.857143,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,902a88ec-2eee-444f-a524-c7f0e5f5c3bc,Drinkware


### Export this to csv 

In [ ]:
data_with_entry_prod_cat.to_csv('data_with_entry_prod_cat.csv')

In [ ]:
import pandas as pd
import numpy as np
from google.cloud.bigquery import Client, QueryJobConfig
import seaborn as sns

In [ ]:
data = pd.read_csv('data_all_merged_03_08_23_vfinal.csv')
data = data.iloc[:,1:]

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """
SELECT *
FROM
  (
  SELECT 
    fo_prod_table.yeti_customer_uuid,
    fo_prod_table.first_order_id, 
    fo_prod_table.price,
    fo_prod_table.product_category,
    DENSE_RANK() OVER(PARTITION BY fo_prod_table.yeti_customer_uuid ORDER BY fo_prod_table.price DESC) as rnk1
  FROM
    (
    SELECT 
      fo_id.yeti_customer_uuid,
      fo_id.first_order_id, 
      oi_prod_table.product_id, 
      oi_prod_table.product_category, 
      oi_prod_table.price,
    FROM
      (SELECT
        orders.yeti_customer_uuid,
        COUNT(orders.order_id) as order_cnt,
        MIN(orders.order_id) as first_order_id,
        MIN(orders.creation_date) first_order_date,
      FROM `yeti-dev-edw.aa_anon.aa_order` orders
      WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
      GROUP BY orders.yeti_customer_uuid
      HAVING order_cnt > 1) fo_id
    LEFT JOIN
      (
      SELECT
        order_item.yeti_customer_uuid,
        order_item.order_id,
        order_item.order_item_id,
        order_item.product_id, 
        product.product_category, 
        product.price, 
      FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
      LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
      ON order_item.product_id = product.product_id) oi_prod_table
    ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
    ) fo_prod_table
  )
WHERE rnk1 = 1
"""

job = client.query(query)
df = job.to_dataframe()

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

query = """
SELECT *
FROM
  (
  SELECT 
    fo_prod_table.yeti_customer_uuid,
    fo_prod_table.first_order_id, 
    fo_prod_table.price,
    fo_prod_table.product_category,
    fo_prod_table.product_spend,
    DENSE_RANK() OVER(PARTITION BY fo_prod_table.yeti_customer_uuid ORDER BY fo_prod_table.product_spend DESC) as rnk1
  FROM
    (
    SELECT 
      fo_id.yeti_customer_uuid,
      fo_id.first_order_id, 
      oi_prod_table.product_id, 
      oi_prod_table.product_category, 
      oi_prod_table.price,
      oi_prod_table.product_spend,
    FROM
      (SELECT
        orders.yeti_customer_uuid,
        COUNT(orders.order_id) as order_cnt,
        MIN(orders.order_id) as first_order_id,
        MIN(orders.creation_date) first_order_date,
      FROM `yeti-dev-edw.aa_anon.aa_order` orders
      WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
      GROUP BY orders.yeti_customer_uuid
      HAVING order_cnt > 1) fo_id
    LEFT JOIN
      (
      SELECT
        order_item.yeti_customer_uuid,
        order_item.order_id,
        order_item.order_item_id,
        order_item.product_id, 
        product.product_category, 
        product.price, 
        order_item.quantity*product.price as product_spend,
      FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
      LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
      ON order_item.product_id = product.product_id) oi_prod_table
    ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
    ) fo_prod_table
  )
WHERE rnk1 = 1
ORDER BY yeti_customer_uuid
"""

job = client.query(query)
df = job.to_dataframe()

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat[data_with_entry_prod_cat['more_than_1_entry_prod_cat?'] == 0]

In [ ]:
df.pivot_table(index = ['yeti_customer_uuid'], aggfunc ='size')


yeti_customer_uuid
0000095c-0e7f-4fe6-917e-1d2356aed1c4    1
00001699-04d3-45e8-aedf-6c2a1cdd57ea    2
00001dcc-dd92-49e9-b2a1-42cf8fd923fa    1
000023e6-98b2-4f5d-bc9d-6232e3eb244e    1
00003f8f-fdc9-4b04-860f-7b6e36bb8af5    1
                                       ..
ffffcdf6-70c6-43a7-b5c0-088eae60a06a    1
ffffe8b5-794e-4537-962a-00e88b920c3e    1
ffffedf7-056d-4fa8-836e-d805d90558fe    1
fffff759-bb94-4ce8-92bd-b156dd29de1b    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

In [ ]:
df[df['yeti_customer_uuid'] == '00001699-04d3-45e8-aedf-6c2a1cdd57ea']

,yeti_customer_uuid,first_order_id,price,product_category,product_spend,rnk1
2,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.0,Drinkware,25.0,1
3,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.0,Drinkware,25.0,1


In [ ]:
entry_product_cat_df = df[df.duplicated()==False]

In [ ]:
entry_product_cat_df

,yeti_customer_uuid,first_order_id,price,entry_prod_cat,product_spend,rnk1
0,None,100003602,NaN,None,NaN,1
1,0000095c-0e7f-4fe6-917e-1d2356aed1c4,INV-571256,38.00,Drinkware,38.00,1
2,00001699-04d3-45e8-aedf-6c2a1cdd57ea,202125188,25.00,Drinkware,25.00,1
4,00001dcc-dd92-49e9-b2a1-42cf8fd923fa,201989165,18.74,Drinkware,18.74,1
5,000023e6-98b2-4f5d-bc9d-6232e3eb244e,202173939,25.00,Drinkware,100.00,1
...,...,...,...,...,...,...
2692502,ffffcdf6-70c6-43a7-b5c0-088eae60a06a,203424722,40.00,Drinkware,40.00,1
2692503,ffffe8b5-794e-4537-962a-00e88b920c3e,203877222,250.00,Hard Cooler,250.00,1
2692504,ffffedf7-056d-4fa8-836e-d805d90558fe,202498224,37.49,Drinkware,899.76,1
2692505,fffff759-bb94-4ce8-92bd-b156dd29de1b,INV-318899,38.00,Drinkware,38.00,1


In [ ]:
data_with_entry_prod_cat.pivot_table(index = ['customer_id'],aggfunc = 'size')

customer_id
0000095c-0e7f-4fe6-917e-1d2356aed1c4    1
00001699-04d3-45e8-aedf-6c2a1cdd57ea    1
00001dcc-dd92-49e9-b2a1-42cf8fd923fa    1
000023e6-98b2-4f5d-bc9d-6232e3eb244e    1
00003f8f-fdc9-4b04-860f-7b6e36bb8af5    1
                                       ..
ffffcdf6-70c6-43a7-b5c0-088eae60a06a    1
ffffe8b5-794e-4537-962a-00e88b920c3e    1
ffffedf7-056d-4fa8-836e-d805d90558fe    1
fffff759-bb94-4ce8-92bd-b156dd29de1b    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

In [ ]:
data_with_entry_prod_cat[data_with_entry_prod_cat['customer_id']=='b7b75c1c-c906-446c-9f8c-f0e899618776']

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat,price
1613862,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Outdoor Living,300.0


In [ ]:
data_with_entry_prod_cat[['customer_id']].value_counts()

customer_id                         
b7b75c1c-c906-446c-9f8c-f0e899618776    3
bfebf243-c72f-4459-bad0-aec00b75a98a    3
a290f9f3-a2ed-49ae-b83a-74faffbcd1ef    3
5539a118-2b3f-4fc3-a47f-5a9081260597    3
a9a337af-f30e-4a37-bf8a-13debca27b13    3
                                       ..
55516972-7b1a-4651-b125-53311a8426d9    1
5551515d-6d7f-4e71-8cb6-975fda97f762    1
555148fe-9d58-4de9-957e-b8b979a58de0    1
5551470e-a7c8-43f8-abcd-01cd280d791c    1
fffffbe6-2958-4c16-a5e2-864a6cb197a4    1
Length: 2131520, dtype: int64

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat,price
1613357,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Soft Cooler,300.0
1613358,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Outdoor Living,300.0
1613359,b7b75c1c-c906-446c-9f8c-f0e899618776,16.1,357188.0,949.968085,0.4,0.0,0.2,0.4,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,1.0,b7b75c1c-c906-446c-9f8c-f0e899618776,Bags,300.0


In [ ]:
# data_with_entry_prod_cat.drop(['more_than_1_entry_prod_cat?','price'],axis=1)

In [ ]:
def make_entry_prod_cat_null(cust_id):
    if cust_id in cust_with_more_than_1_entry_prod_cat_list:
        return 1
    else:
        return 0

In [ ]:
data_with_entry_prod_cat['more_than_1_entry_prod_cat?']=data_with_entry_prod_cat['customer_id'].map(make_entry_prod_cat_null)

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat[data_with_entry_prod_cat['more_than_1_entry_prod_cat?'] == 0]

In [ ]:
# data_with_entry_prod_cat.drop(['more_than_1_entry_prod_cat?','price'],axis=1)

In [ ]:
data_with_entry_prod_cat = data_with_entry_prod_cat.iloc[:,:-2] # dropping more_than_1_entry_prod_cat? and price column

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].value_counts()

Drinkware         1618315
Hard Cooler        210632
Soft Cooler        113937
Other               60593
Bags                47296
Outdoor Living      38465
Cargo               14980
(not set)            2730
Name: entry_prod_cat, dtype: int64

In [ ]:
data_with_entry_prod_cat.columns

Index(['customer_id', 'months_elapsed', 'lifetime_spent', 'per_order_spent',
       'outdoor_equipment_pct', 'Cargo_pct', 'Soft_Cooler_pct', 'Bags_pct',
       'drinkware_pct', 'Hard_Cooler_pct', 'other_pct', 'purchase_times',
       'recency', 'avg_time_btw_orders', 'email_received', 'email_open_rate',
       'unsubscribed', 'months_since_last_email_open',
       'num_distinct_prod_cats', 'lifetime_quantity', 'per_order_quantity',
       'has_discount', 'black', 'navy', 'white', 'seafoam', 'stainless',
       'charcoal', 'nordic_purple', 'harvest_red', 'alpine_yellow',
       'other_color', 'yeti_customer_uuid', 'entry_prod_cat'],
      dtype='object')

In [ ]:
def convert_entry_prod_other_cats(prod_cat):
    if prod_cat == '(not set)':
        return 'Other'
    else:
        return prod_cat

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].map(convert_entry_prod_other_cats).value_counts()

Drinkware         1618315
Hard Cooler        210632
Soft Cooler        113937
Other               63323
Bags                47296
Outdoor Living      38465
Cargo               14980
Name: entry_prod_cat, dtype: int64

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].value_counts().sum()

2106948

In [ ]:
data_with_entry_prod_cat['entry_prod_cat'].isna().sum()

20307

In [ ]:
data_with_entry_prod_cat

,customer_id,months_elapsed,lifetime_spent,per_order_spent,outdoor_equipment_pct,Cargo_pct,Soft_Cooler_pct,Bags_pct,drinkware_pct,Hard_Cooler_pct,...,white,seafoam,stainless,charcoal,nordic_purple,harvest_red,alpine_yellow,other_color,yeti_customer_uuid,entry_prod_cat
0,b32e89d5-a934-4d77-8b86-e01bf270fce6,83.700000,1771002.0,124.806342,0.000000,0.0,0.000000,0.000000,0.974359,0.025641,...,5.0,1.0,5.0,0.0,0.0,0.0,0.0,19.0,b32e89d5-a934-4d77-8b86-e01bf270fce6,Drinkware
1,20a5cbb9-216a-4bfb-84f8-34316cc990f0,64.566667,766093.0,194.637449,0.000000,0.0,0.055556,0.000000,0.944444,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,20a5cbb9-216a-4bfb-84f8-34316cc990f0,Drinkware
2,564fd988-bc35-475f-b7f6-da04ae474422,56.066667,910602.0,91.288421,0.047619,0.0,0.000000,0.000000,0.619048,0.047619,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,18.0,564fd988-bc35-475f-b7f6-da04ae474422,Drinkware
3,e6867db2-26c5-4841-8822-863bce76f5b0,69.266667,1137507.0,84.787343,0.125000,0.0,0.000000,0.000000,0.437500,0.000000,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,20.0,e6867db2-26c5-4841-8822-863bce76f5b0,Drinkware
4,36a6c5a8-955a-457e-8509-83b7e0644856,23.666667,2138149.0,386.995294,0.000000,0.0,0.176471,0.058824,0.588235,0.000000,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,9.0,36a6c5a8-955a-457e-8509-83b7e0644856,Drinkware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135824,c6882f0e-203b-41f7-bc5f-1f24713a3179,47.566667,3749.0,178.523810,0.000000,0.0,0.000000,0.000000,0.857143,0.142857,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,c6882f0e-203b-41f7-bc5f-1f24713a3179,Hard Cooler
2135825,1cb47d1e-ead0-4432-8d40-53ce0d3df887,86.700000,357405.0,120.826572,0.000000,0.0,0.000000,0.000000,0.857143,0.142857,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,1cb47d1e-ead0-4432-8d40-53ce0d3df887,Drinkware
2135826,3973bf5a-f1db-4a38-8ffb-bfd8a46a5027,58.966667,667411.0,64.728057,0.000000,0.0,0.000000,0.000000,0.857143,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,3973bf5a-f1db-4a38-8ffb-bfd8a46a5027,Drinkware
2135827,902a88ec-2eee-444f-a524-c7f0e5f5c3bc,35.066667,243385.0,69.261525,0.000000,0.0,0.000000,0.000000,0.857143,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,902a88ec-2eee-444f-a524-c7f0e5f5c3bc,Drinkware


### Export this to csv 

In [ ]:
data_with_entry_prod_cat.to_csv('data_with_entry_prod_cat.csv')